In [4]:
import os


import matplotlib.pyplot as plt
import xarray as xr

In [10]:
DATA_PATH = '/g/data/ua8/LE_models/20CRv3/mean_daily/prate'

In [36]:
from dask.distributed import Client
client = Client(threads_per_worker=1, n_workers=12)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 12
Total threads: 12,Total memory: 251.18 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:38509,Workers: 12
Dashboard: /proxy/8787/status,Total threads: 12
Started: Just now,Total memory: 251.18 GiB
Comm: tcp://127.0.0.1:33667,Total threads: 1
Dashboard: /proxy/38817/status,Memory: 20.93 GiB
Nanny: tcp://127.0.0.1:46801,


2024-06-07 22:36:24,279 - distributed.nanny - WARNING - Restarting worker
2024-06-07 22:36:24,283 - distributed.nanny - WARNING - Restarting worker
2024-06-07 22:36:24,291 - distributed.nanny - WARNING - Restarting worker
2024-06-07 22:36:24,298 - distributed.nanny - WARNING - Restarting worker
2024-06-07 22:36:24,306 - distributed.nanny - WARNING - Restarting worker
2024-06-07 22:36:24,313 - distributed.nanny - WARNING - Restarting worker
2024-06-07 22:36:24,320 - distributed.nanny - WARNING - Restarting worker
2024-06-07 22:36:24,327 - distributed.nanny - WARNING - Restarting worker
2024-06-07 22:36:24,332 - distributed.nanny - WARNING - Restarting worker
2024-06-07 22:36:24,336 - distributed.nanny - WARNING - Restarting worker
2024-06-07 22:36:24,344 - distributed.nanny - WARNING - Restarting worker
2024-06-07 22:36:24,350 - distributed.nanny - WARNING - Restarting worker


In [11]:
os.listdir(DATA_PATH)[:5]

['prate.1842.nc',
 'prate.1909.nc',
 'prate.1861.nc',
 'prate.1991.nc',
 'prate.1953.nc']

In [40]:
def __preprocess_20CR(ds):
    return ds['prate']

In [57]:
paths = os.listdir(DATA_PATH)[:15]

In [58]:
paths = os.path.join(DATA_PATH, '*.nc')
paths

'/g/data/ua8/LE_models/20CRv3/mean_daily/prate/*.nc'

In [65]:
precip_ds = xr.open_mfdataset(paths, preprocess=__preprocess_20CR).chunk({'time':-1, 'lon':30, 'lat':18})

In [66]:
precip_ds

<xarray.Dataset> Size: 17GB
Dimensions:  (lat: 181, lon: 360, time: 65744)
Coordinates:
  * lat      (lat) float32 724B -90.0 -89.0 -88.0 -87.0 ... 87.0 88.0 89.0 90.0
  * lon      (lon) float32 1kB 0.0 1.0 2.0 3.0 4.0 ... 356.0 357.0 358.0 359.0
  * time     (time) datetime64[ns] 526kB 1836-01-01 1836-01-02 ... 2015-12-31
Data variables:
    prate    (time, lat, lon) float32 17GB dask.array<chunksize=(65744, 18, 30), meta=np.ndarray>

In [71]:
import flox.xarray

In [ ]:
#https://flox.readthedocs.io/en/latest/user-stories/climatology-hourly.html
# flox.xarray.xarray_reduce(ds.tp, ds.time.dt.hour, func="mean")

In [74]:
rx1d_ds_flox = flox.xarray.xarray_reduce(precip_ds.prate, precip_ds.time.dt.year, func='max')

In [75]:
rx1d_ds_flox = rx1d_ds_flox.compute()

Task exception was never retrieved
future: <Task finished name='Task-256913' coro=<Client._gather.<locals>.wait() done, defined at /g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/distributed/client.py:2197> exception=AllExit()>
Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/distributed/client.py", line 2206, in wait
    raise AllExit()
distributed.client.AllExit
Task exception was never retrieved
future: <Task finished name='Task-256917' coro=<Client._gather.<locals>.wait() done, defined at /g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/distributed/client.py:2197> exception=AllExit()>
Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/distributed/client.py", line 2206, in wait
    raise AllExit()
distributed.client.AllExit
Task exception was never retrieved

In [68]:
rx1d_ds = precip_ds.resample(time='YE').max().compute()

Task exception was never retrieved
future: <Task finished name='Task-109342' coro=<Client._gather.<locals>.wait() done, defined at /g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/distributed/client.py:2197> exception=AllExit()>
Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/distributed/client.py", line 2206, in wait
    raise AllExit()
distributed.client.AllExit
/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/distributed/client.py:3161: UserWarning: Sending large graph of size 12.35 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


In [70]:
rx1d_ds.prate.

<xarray.Dataset> Size: 47MB
Dimensions:  (time: 180, lat: 181, lon: 360)
Coordinates:
  * lat      (lat) float32 724B -90.0 -89.0 -88.0 -87.0 ... 87.0 88.0 89.0 90.0
  * lon      (lon) float32 1kB 0.0 1.0 2.0 3.0 4.0 ... 356.0 357.0 358.0 359.0
  * time     (time) datetime64[ns] 1kB 1836-12-31 1837-12-31 ... 2015-12-31
Data variables:
    prate    (time, lat, lon) float32 47MB 5.125e-06 5.125e-06 ... 9.45e-05